In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [2]:
df = pd.read_csv('cancer_data.csv')

In [3]:
df.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,0,5,1,1,1,2,1,3,1,1
1,0,5,4,4,5,7,10,3,2,1
2,0,3,1,1,1,2,2,3,1,1
3,0,6,8,8,1,3,4,3,7,1
4,0,4,1,1,3,2,1,3,1,1


In [4]:
df.describe()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
count,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,0.349927,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221
std,0.477296,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000
50%,0.000000,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000
75%,1.000000,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000
max,1.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [5]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
Class,1.000000,0.714790,0.820801,0.821891,0.706294,0.690958,0.822696,0.758228,0.718677,0.423448
age,0.714790,1.000000,0.642481,0.653470,0.487829,0.523596,0.593091,0.553742,0.534066,0.350957
menopause,0.820801,0.642481,1.000000,0.907228,0.706977,0.753544,0.691709,0.755559,0.719346,0.460755
tumor-size,0.821891,0.653470,0.907228,1.000000,0.685948,0.722462,0.713878,0.735344,0.717963,0.441258
inv-nodes,0.706294,0.487829,0.706977,0.685948,1.000000,0.594548,0.670648,0.668567,0.603121,0.418898
node-caps,0.690958,0.523596,0.753544,0.722462,0.594548,1.000000,0.585716,0.618128,0.628926,0.480583
deg-malig,0.822696,0.593091,0.691709,0.713878,0.670648,0.585716,1.000000,0.680615,0.584280,0.339210
breast,0.758228,0.553742,0.755559,0.735344,0.668567,0.618128,0.680615,1.000000,0.665602,0.346011
breast-quad,0.718677,0.534066,0.719346,0.717963,0.603121,0.628926,0.584280,0.665602,1.000000,0.433757
irradiat,0.423448,0.350957,0.460755,0.441258,0.418898,0.480583,0.339210,0.346011,0.433757,1.000000


In [6]:
endog = df['tumor-size']
exog = sm.add_constant(df[['Class', 'breast', 'node-caps']])
print(exog)

     const  Class  breast  node-caps
0      1.0      0       3          2
1      1.0      0       3          7
2      1.0      0       3          2
3      1.0      0       3          3
4      1.0      0       3          2
..     ...    ...     ...        ...
678    1.0      0       1          3
679    1.0      0       1          2
680    1.0      1       8          7
681    1.0      1      10          3
682    1.0      1      10          4

[683 rows x 4 columns]


In [7]:
x = exog.to_numpy()
y = endog.to_numpy()
s1_xt = np.transpose(x)
print(s1_xt)

[[ 1.  1.  1. ...  1.  1.  1.]
 [ 0.  0.  0. ...  1.  1.  1.]
 [ 3.  3.  3. ...  8. 10. 10.]
 [ 2.  7.  2. ...  7.  3.  4.]]


In [8]:
s2_mul1 = np.matmul(s1_xt, x)
print(s2_mul1)

[[  683.   239.  2353.  2209.]
 [  239.   239.  1428.  1273.]
 [ 2353.  1428. 12199.  9906.]
 [ 2209.  1273.  9906. 10515.]]


In [9]:
s3_inv = np.linalg.inv(s2_mul1)
print(s3_inv)

[[ 0.00643412  0.0048405  -0.00099655 -0.00099886]
 [ 0.0048405   0.01864885 -0.00194711 -0.00144028]
 [-0.00099655 -0.00194711  0.00059872 -0.00011896]
 [-0.00099886 -0.00144028 -0.00011896  0.00059138]]


In [10]:
s4_mul = np.matmul(s3_inv, s1_xt)
print(s4_mul)

[[ 1.44672899e-03 -3.54759596e-03  1.44672899e-03 ... -3.68986849e-03
  -1.68751835e-03 -2.68638334e-03]
 [-3.88140178e-03 -1.10827988e-02 -3.88140178e-03 ... -2.16952056e-03
  -3.02632966e-04 -1.74291236e-03]
 [ 5.61685593e-04 -3.31021259e-05  5.61685593e-04 ...  1.01337552e-03
   2.68664276e-03  2.56768521e-03]
 [-1.72977400e-04  2.78392315e-03 -1.72977400e-04 ...  7.48856038e-04
  -1.85457949e-03 -1.26319938e-03]]


In [11]:
s5_res = np.matmul(s4_mul, y)
print(s5_res)

[0.15673213 3.04707394 0.25198575 0.3475685 ]


In [13]:
#build MLR model using OLS method
mod = sm.OLS(endog, exog)
results = mod.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    ESE   R-squared:                       0.581
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     31.92
Date:                Sun, 19 Dec 2021   Prob (F-statistic):           4.68e-13
Time:                        15:31:39   Log-Likelihood:                -209.26
No. Observations:                  73   AIC:                             426.5
Df Residuals:                      69   BIC:                             435.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         31.1156      5.154      6.037      0.0

In [12]:
#MLR using scikit 
from sklearn import linear_model
x = df[['Class', 'breast', 'node-caps']]
y = df['tumor-size']

lm = linear_model.LinearRegression()
model = lm.fit(x, y)
lm.coef_

array([3.04707394, 0.25198575, 0.3475685 ])

In [13]:
lm.intercept_

0.15673213301699018

In [14]:
lm.coef_

array([3.04707394, 0.25198575, 0.3475685 ])